In [23]:
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization

In [16]:
num_classes = 2
img_rows, img_cols = 50,50
batch_size = 16

train_data_dir = './Data/train/'
validation_data_dir = './Data/val/'

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 40,
                                  width_shift_range = .3,
                                  height_shift_range = .3,
                                  shear_range = .2,
                                  zoom_range = .2,
                                  horizontal_flip = True,
                                  fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale = 1.0 / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(img_rows, img_cols),
                                                   batch_size = batch_size,
                                                   class_mode = 'binary',
                                                   shuffle = True)

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                              target_size = (img_rows, img_cols),
                                                              batch_size = batch_size,
                                                              class_mode = 'binary',
                                                              shuffle = False)

Found 2966 images belonging to 2 classes.
Found 742 images belonging to 2 classes.


In [17]:
from keras.optimizers import RMSprop, SGD

input_shape = (50,50,3)

model = Sequential()

model.add(Conv2D(32, 
                 kernel_size=(3,3),
                 activation='relu',
                 input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(64,
                 (3,3),
                 activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())

model.add(Dropout(.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer = 'adam',
             metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
batch_normalization_3 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 46, 64)        18496     
_________________________________________________________________
batch_normalization_4 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 33856)            

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('./Model/mask_detection_model.h5',
                             monitor = 'val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss',
                         min_delta= 0,
                         patience = 20,
                         verbose=1,
                         restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=.2,
                             patience=5,
                             verbose = 1,
                             min_delta = .0001)

callbacks = [earlystop, reduce_lr, checkpoint]


no_train = 2966
no_val = 742
epochs = 100

In [19]:
history = model.fit(train_generator,
                    steps_per_epoch = no_train // batch_size,
                    epochs  = epochs,
                    callbacks = callbacks,
                    validation_data = validation_generator,
                    validation_steps=no_val // batch_size)

Epoch 1/100
185/185 [==============================] - 62s 323ms/step - loss: 0.4149 - accuracy: 0.8542 - val_loss: 0.5407 - val_accuracy: 0.6413

Epoch 00001: val_loss improved from inf to 0.54066, saving model to ./Model\mask_detection_model.h5
Epoch 2/100
185/185 [==============================] - 52s 281ms/step - loss: 0.2867 - accuracy: 0.8822 - val_loss: 0.2036 - val_accuracy: 0.9321

Epoch 00002: val_loss improved from 0.54066 to 0.20361, saving model to ./Model\mask_detection_model.h5
Epoch 3/100
185/185 [==============================] - 44s 240ms/step - loss: 0.2667 - accuracy: 0.8982 - val_loss: 0.1635 - val_accuracy: 0.9497

Epoch 00003: val_loss improved from 0.20361 to 0.16354, saving model to ./Model\mask_detection_model.h5
Epoch 4/100
185/185 [==============================] - 46s 251ms/step - loss: 0.2364 - accuracy: 0.9106 - val_loss: 0.0795 - val_accuracy: 0.9701

Epoch 00004: val_loss improved from 0.16354 to 0.07950, saving model to ./Model\mask_detection_model.h5
